# A simplistic Vector class

A very simplistic vector class for some linear algebra experimentation

In [37]:
import math

class Vector(object):
    def __init__(self, coordinates):
        try:
            if not coordinates:
                raise ValueError
            self.coordinates = coordinates
            self.dimensions = len(coordinates)

        except ValueError:
            raise ValueError("The coordinates must be non empty")

        except TypeError:
            raise TypeError("The coordinates must be an iterable")

    def __getitem__(self, i):
        return self.coordinates[i]
    
    def __setitem__(self, i, value):
        self.coordinates[i] = value
    
    def __str__(self):
        return 'Vector {}'.format(self.coordinates)

    def __eq__(self, other):
        return self.coordinates == other.coordinates
    
    def __len__(self):
        return self.dimensions
    
    def __add__(self, other):
        if len(self) != len(other):
            raise ArithmeticError("Not  matching dimensions")
        sum = [x+y for x, y in zip(self.coordinates, other.coordinates)]
        return Vector(sum)
        
    def __sub__(self, other):
        if len(self) != len(other):
            raise ArithmeticError("Not  matching dimensions")
        diff = [x-y for x, y in zip(self.coordinates, other.coordinates)]
        return Vector(diff)
    
    def __rmul__(self, other):
        if not isinstance(other, float):
            raise ArithmeticError("Not  matching dimensions")
        product = [x * other for x in self.coordinates]
        return Vector(product)
    
    def magnitude(self):
        magnitude_squared = [x**2 for x in self.coordinates];
        return sum(magnitude_squared)**(.5)
    
    def normalize(self):
        try:
            magnitude = self.magnitude()
            return (1./magnitude) * self
        except ZeroDivisionError:
            raise Exception('Cannot normalize the zero vector')
            
    def dot_prod(self, other):
        product = [x*y for x, y in zip(self.coordinates, other.coordinates)]
        sum = 0
        for v in product:
            sum += v;
        return sum
        
    def angle(self, other, is_degree = False):
        if is_degree:
            return angle_grad(self, other)
        return math.acos((self.dot_prod(other))
                         /(self.magnitude() * other.magnitude()))
    
    def angle_grad(self, other):
        return self.angle(other) * (180.0 / math.pi)
    
    def is_orthogonal_to(self, other, tolerance = 1e-10):
        return abs(self.dot_prod(other)) < tolerance
    
    def is_parallel_to(self, other):
        return (self.is_zero() or other.is_zero() or \
            self.angle(other) == 0 or \
            self.angle(other) == pi)
                
    def is_zero(self, tolerance = 1e-10):
        return self.magnitude() < tolerance
    
    def component_parallel_to(self, basis):
        
        if basis.is_zero():
            raise Exception("There is not unique parallel component to the zero vector")
            
        # Calculating unit vector
        u = basis.normalize()
        weight = self.dot_prod(u)
        return (weight *  u)
    
    def component_orthogonal_to(self, basis):
        
        if basis.is_zero():
            raise Exception("There is not unique orthogonal component to the zero vector")
            
        projection = self.component_parallel_to(basis)
        return (self - projection)

### Basic behavior

In [2]:
Vector([1,2,3,4]) == Vector([1,2,3,4])

True

In [3]:
Vector([1,2,3,4]) == Vector([-1,2,3,4])

False

In [4]:
print(Vector([3,4,5]))

Vector [3, 4, 5]


### Addition, subtraction and multiplication

In [5]:
print(Vector([8.218, -9.341]) + Vector([-1.129, 2.111]))

Vector [7.089, -7.229999999999999]


In [6]:
print(Vector([7.119, 8.215]) - Vector([-8.223, 0.878]))

Vector [15.342, 7.337]


In [7]:
print(7.41 * Vector([1.671, -1.012, -0.318]))

Vector [12.38211, -7.49892, -2.35638]


### Magnitude and Direction

In [8]:
Vector([1,2,3,4]).magnitude()

5.477225575051661

Direction represented as unit vector (after normalization)

In [9]:
print(Vector([-1,1,1]).normalize())

Vector [-0.5773502691896258, 0.5773502691896258, 0.5773502691896258]


Now, the magnitude of the unit vector should be the unit vector of length 1

In [10]:
v = Vector([-1,1,1]).normalize()
v.magnitude()

1.0

### Dot Product (Inner Product) and Angle

The Dot Product of v1 and v2  satisified following identity:

$$ \vec{v} * \vec{w} = \|\vec{v}\| * \|\vec{w}\| * cos\theta$$

so

$$ \theta = arccos(\frac{\vec{v} * \vec{w}}{\|\vec{v}\| * \|\vec{w}\| })$$

In [11]:
v = Vector([1,2,-1])
w = Vector([3,1,0])
prod = v.dot_prod(w)
print(prod)

5


In [12]:
v.angle(w)

0.8691222030072928

In [13]:
v.angle_grad(w)

49.79703411343022

And we check the identity

In [14]:
v.dot_prod(w) == v.magnitude() * w.magnitude() * math.cos(v.angle(w))

True

### Parallel and orthogonal Vectors

We check our implementation with a simple vector with a zero x, respectively y component

In [15]:
v1 = Vector([1,0])
v2 = Vector([0,1])
v1.is_orthogonal_to(v2)

True

The zero vector is orthogonal to itself as well

In [16]:
v0 = Vector([0,0])
v0.is_orthogonal_to(v0)

True

Next, we validate our is_parallel_to function. Any vector is parallel to itself.

In [17]:
v1 = Vector([4,5])
v1.is_parallel_to(v1)

True

The zero vector is not only orthogonal, but parallel to itself as well.

In [18]:
v0 = Vector([0,0])
v0.is_parallel_to(v0)

True

But more than that, the zero vector is also parallel to any other vector.

In [19]:
v0.is_parallel_to(Vector([5,6]))

True

### Projecting Vectors
Projecting vectors onto other vectors and calculating their orthogonal and parallel component

In [38]:
v1 = Vector([3.039, 1.879])
v2 = Vector([0.825, 2.036])
v = v1.component_parallel_to(v2)
print(v)

Vector [1.082606962484467, 2.671742758325303]


In [39]:
v1 = Vector([-9.88, -3.264, -8.159])
v2 = Vector([-2.155, -9.353, -9.473])
v = v1.component_orthogonal_to(v2)
print(v)

Vector [-8.350081043195763, 3.376061254287722, -1.4337460427811841]
